In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from kilosort import run_kilosort, io
import spikeinterface.extractors as se

### Load Configs

In [11]:
from config import dir_config, main_config, s_probe_path

raw_dir = Path(dir_config.data.raw)
sorting_dir = Path(dir_config.data.sorting)
compiled_dir = Path(dir_config.data.compiled)

session_metadata = pd.read_csv(Path(compiled_dir, "sessions_metadata.csv"), index_col=None)

### Utils functions

In [12]:

def convert_nsx_to_bin(nsx_path, nsx_name, bin_path, bin_name):
    nsx_data, N, c, s, fs, probe_path = None, None, None, None, None, None
    
    nsx_file = Path(nsx_path, f'{nsx_name}.ns5')
    if nsx_file.exists():
        nsx_data = se.read_blackrock(file_path=Path(nsx_path, nsx_name), stream_id='5')   
        filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
        nsx_data, bin_path, data_name=f"{bin_name}.bin", dtype=np.int16,
        chunksize=60000, export_probe=True, probe_name='probe.prb'
        )
        print(f"{N}, {c}, {s}, {fs}, {probe_path}")
    else:
        print(f"File {nsx_file} does not exist")
    return nsx_data, N, c, s, fs, probe_path

## Covert blackrock (NSX) data to bin

In [13]:
for idx, session_id in enumerate(session_metadata['session_id']):
    # Construct the path to the raw and binary file
    nsx_path = Path(raw_dir) / session_id
    nsx_name = session_id
    bin_path = Path(sorting_dir) / session_id
    bin_name = session_id
    bin_file = bin_path / f"{bin_name}.bin"
    
    try:
        # check if raw data exists
        if not nsx_path.exists():
            print(f"Raw data does not exist for: \t {session_id}")
            continue
        
        # Check if the binary file already exists
        if not bin_file.exists():
            print(f"Converting {session_id} to binary")            
            # Call the conversion function
            nsx_data, N, c, s, fs, probe_path = convert_nsx_to_bin(nsx_path=str(nsx_path), nsx_name=nsx_name, bin_path=str(bin_path), bin_name=bin_name)
        else:
            print(f"Bin file exists for: \t {bin_name}")
    except Exception as e:
        print(f"Error: {e}")
        continue

Bin file exists for: 	 210126_GP_JP
Bin file exists for: 	 210205_GP_JP
Bin file exists for: 	 210210_GP_JP
Bin file exists for: 	 210211_GP_JP
Bin file exists for: 	 210216_GP_JP
Bin file exists for: 	 210217_GP_JP
Bin file exists for: 	 210224_GP_JP
Bin file exists for: 	 210302_GP_JP
Bin file exists for: 	 210305_GP_JP
Bin file exists for: 	 210311_GP_JP
Bin file exists for: 	 210312_GP_JP
Bin file exists for: 	 210315_GP_JP
Bin file exists for: 	 210316_GP_JP
Bin file exists for: 	 210317_GP_JP
Bin file exists for: 	 210318_GP_JP
Bin file exists for: 	 210323_GP_JP
Bin file exists for: 	 210413_GP_JP
Bin file exists for: 	 210524_GP_JP
Bin file exists for: 	 210525_GP_JP
Bin file exists for: 	 210528_GP_JP
Bin file exists for: 	 210601_GP_JP
Bin file exists for: 	 210602_GP_JP
Bin file exists for: 	 210603_GP_JP
Bin file exists for: 	 210608_GP_JP
Bin file exists for: 	 210609_GP_JP
Bin file exists for: 	 210708_GP_JP
Bin file exists for: 	 240625_GP_TZ
Bin file exists for: 	 24062

kilosort.io : ========================================
kilosort.io : Loading recording with SpikeInterface...
kilosort.io : number of samples: 188698650
kilosort.io : number of channels: 19
kilosort.io : numbef of segments: 1
kilosort.io : sampling rate: 30000.0
kilosort.io : dtype: int16
kilosort.io : ========================================
kilosort.io : Converting 3145 data chunks with a chunksize of 60000 samples...
kilosort.io : 179 of 3145 chunks converted...
kilosort.io : 321 of 3145 chunks converted...
kilosort.io : 460 of 3145 chunks converted...
kilosort.io : 595 of 3145 chunks converted...
kilosort.io : 726 of 3145 chunks converted...
kilosort.io : 852 of 3145 chunks converted...
kilosort.io : 976 of 3145 chunks converted...
kilosort.io : 1099 of 3145 chunks converted...
kilosort.io : 1216 of 3145 chunks converted...
kilosort.io : 1331 of 3145 chunks converted...
kilosort.io : 1447 of 3145 chunks converted...
kilosort.io : 1557 of 3145 chunks converted...
kilosort.io : 1667 

188698650, 19, 1, 30000.0, None


## Run Kilosort

#### KS settings

In [14]:
drift_correction = True

settings_s_probe_ucla = {
    'fs': 30000,
    'batch_size': 180000,
    'min_template_size': 30,
    'nearest_templates': 6,
    'n_chan_bin': 18,
}
settings_s_probe_uw = {
    'fs': 30000,
    'batch_size': 180000,
    'min_template_size': 30,
    # 'dmin': 100,# might be helpful to center template at contacts, not between contacts
    'nearest_templates': 10,
    'n_chan_bin': 19, 
    'Th_universal': 9,
    'Th_learned': 8,
    
}
if drift_correction:
    settings_s_probe_ucla['nblocks'] = 1
    settings_s_probe_uw['nblocks'] = 1
else:
    settings_s_probe_ucla['nblocks'] = 0
    settings_s_probe_uw['nblocks'] = 0


s_probe = io.load_probe(s_probe_path)
assert s_probe is not None, 'No probe information exported by SpikeInterface'



In [15]:
#     'n_chan_bin': 19,
#     'fs': 30000,
#     'batch_size': 240000,
#     'nblocks': 1,
#     'Th_universal': 9,
#     'Th_learned': 8,
#     'tmin': 0,
#     'tmax': inf,
#     'nt': 61,
#     'shift': None,
#     'scale': None,
#     'artifact_threshold': inf,
#     'nskip': 25,
#     'whitening_range': 32,
#     'highpass_cutoff': 300,
#     'binning_depth': 5,
#     'sig_interp': 20,
#     'drift_smoothing': [0.5, 0.5, 0.5],
#     'nt0min': 20,
#     'dmin': None,
#     'dminx': 32,
#     'min_template_size': 50,
#     'template_sizes': 5,
#     'nearest_chans': 10,
#     'nearest_templates': 6,
#     'max_channel_distance': None,
#     'templates_from_data': True,
#     'n_templates': 6,
#     'n_pcs': 6,
#     'Th_single_ch': 6,
#     'acg_threshold': 0.2,
#     'ccg_threshold': 0.25,
#     'cluster_downsampling': 20,
#     'x_centers': None,
#     'duplicate_spike_ms': 0.25,
#     'filename': WindowsPath('Z:/BassoLabShare/Monkey Studies/SC Priors/data/sorting/241002_GP_TZ/241002_GP_TZ.bin'),
#     'data_dir': WindowsPath('Z:/BassoLabShare/Monkey Studies/SC Priors/data/sorting/241002_GP_TZ'),
#     'data_dtype': 'int16',
#     'do_CAR': True,
#     'invert_sign': False,
#     'NTbuff': 240122,
#     'Nchan': 16,
#     'duplicate_spike_bins': 7,
#     'torch_device': 'cpu',
#     'save_preprocessed_copy': False,
#     'chanMap': array([ 0,  2,  4,  6,  8, 10, 12, 14,  1,  3,  5,  7,  9, 11, 13, 15]),
#     'xc': array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5,
#        7.5, 7.5, 7.5], dtype=float32),
#     'yc': array([  7.5,  57.5, 107.5, 157.5, 207.5, 257.5, 307.5, 357.5, 407.5,
#        457.5, 507.5, 557.5, 607.5, 657.5, 707.5, 757.5], dtype=float32),
#     'kcoords': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#       dtype=float32),
#     'n_chan': 16

In [16]:


for idx, session_id in enumerate(session_metadata['session_id']):
    try:
        session_path = Path(sorting_dir, session_id)
        bin_file = f"{str(session_path)}/{session_id}.bin"
        if session_path.exists():
            # importing correct settings and probe
            if session_metadata['probe'][idx] == 's-probe':
                subject_id = session_metadata['subject'][idx]
                if subject_id == 'Jasper':
                    settings = settings_s_probe_ucla
                    probe = s_probe
                elif subject_id == 'Taz':
                    settings = settings_s_probe_uw
                    probe = s_probe
                    
                # run kilosort
                if not Path(session_path,'kilosort4').exists():
                    settings
                    ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = run_kilosort(
                        settings=settings, probe=probe, filename=bin_file,
                    )
                else:
                    print(f"Already processed with kilosort: {session_id}")
                    
            elif session_metadata['probe'][idx] == 'single':
                print(f"Single electrode probe won't be processed with kilosort: {session_id}")
        else:
            print(f"Session not found: {session_id}")
            continue   

    except Exception as e:
        print(f"Error in {session_id}: {e}")
        continue

Single electrode probe won't be processed with kilosort: 210126_GP_JP
Single electrode probe won't be processed with kilosort: 210205_GP_JP
Single electrode probe won't be processed with kilosort: 210210_GP_JP
Single electrode probe won't be processed with kilosort: 210211_GP_JP
Single electrode probe won't be processed with kilosort: 210216_GP_JP
Single electrode probe won't be processed with kilosort: 210217_GP_JP
Single electrode probe won't be processed with kilosort: 210224_GP_JP
Single electrode probe won't be processed with kilosort: 210302_GP_JP
Single electrode probe won't be processed with kilosort: 210305_GP_JP
Single electrode probe won't be processed with kilosort: 210311_GP_JP
Single electrode probe won't be processed with kilosort: 210312_GP_JP
Single electrode probe won't be processed with kilosort: 210315_GP_JP
Single electrode probe won't be processed with kilosort: 210316_GP_JP
Single electrode probe won't be processed with kilosort: 210317_GP_JP
Single electrode pro

kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Kilosort version 4.0.13
kilosort.run_kilosort: Sorting Z:\BassoLabShare\Monkey Studies\SC Priors\data\sorting\250109_GP_TZ\250109_GP_TZ.bin
kilosort.run_kilosort: Sorting Z:\BassoLabShare\Monkey Studies\SC Priors\data\sorting\250109_GP_TZ\250109_GP_TZ.bin
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort: Using CPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using CPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing prepr

Already processed with kilosort: 241130_GP_TZ
Already processed with kilosort: 241209_GP_TZ
Already processed with kilosort: 241211_GP_TZ
Already processed with kilosort: 241216_GP_TZ
Already processed with kilosort: 241217_GP_TZ
Already processed with kilosort: 241223_GP_TZ
Already processed with kilosort: 241227_GP_TZ
Already processed with kilosort: 241230_GP_TZ
Already processed with kilosort: 250108_GP_TZ


kilosort.run_kilosort: N samples: 188698650
kilosort.run_kilosort: N seconds: 6289.955
kilosort.run_kilosort: N seconds: 6289.955
kilosort.run_kilosort: N batches: 1049
kilosort.run_kilosort: N batches: 1049
kilosort.run_kilosort: Preprocessing filters computed in  10.16s; total  10.17s
kilosort.run_kilosort: Preprocessing filters computed in  10.16s; total  10.17s
kilosort.run_kilosort:  
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: ----------------------------------------
kilosort.spikedetect: Re-computing universal templates from data.
kilosort.spikedetect: Re-computing universal templates from data.
100%|██████████| 1049/1049 [19:02<00:00,  1.09s/it]
kilosort.run_kilosort: drift computed in  1210.52s; total  1220.70s
kilosort.run_kilosort: drift computed in  1210.52s; total  1220.70s
kilosort.run_kilosort:  
kilosor